# 8A. Song Lyrics Generator

In this lab, you will scrape a website to get lyrics of songs by your favorite artist. Then, you will train a model called a Markov chain on these lyrics so that you can generate a song in the style of your favorite artist.

# Question 1. Scraping Song Lyrics

Find a web site that has lyrics for several songs by your favorite artist. Scrape the lyrics into a Python list called `lyrics`, where each element of the list represents the lyrics of one song.

**Tips:**
- Find a web page that has links to all of the songs, like [this one](http://www.azlyrics.com/n/nirvana.html). [_Note:_ It appears that `azlyrics.com` blocks web scraping, so you'll have to find a different lyrics web site.] Then, you can scrape this page, extract the hyperlinks, and issue new HTTP requests to each hyperlink to get each song. 
- Use `time.sleep()` to stagger your HTTP requests so that you do not get banned by the website for making too many requests.

In [1]:
import requests
import time

from bs4 import BeautifulSoup

In [2]:
import spotipy
spotify = spotipy.Spotify()
import sys
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import lxml
import spotfuncs 
import requests
import pandas as pd
import numpy as np
from multiprocessing import Pool

In [3]:
import nltk
from nltk.tokenize import TweetTokenizer

In [65]:
def clean_lyrics(lyrics):
    a_list = lyrics.split("\n")
    new_lyrics = []
    counter = 0
    for i in a_list:
        if i[0] == "[" and i[-1] != "]":
            counter = 1
        if i[0] == "]":
            counter = 0
        if (i[0] != "]" and i[0] != "[") and counter != 1:
            i = i.replace("(","")
            i = i.replace(")","")
            if i != "":
                new_lyrics.append(i)
    return " <N> ".join(new_lyrics)

In [7]:
filepath = "/Users/ramanyakkala/Spec/genius.txt"

In [8]:
artist_id = spotfuncs.search_genius("Travis Scott", credentials_file=filepath)
genius_songs = spotfuncs.get_songs(artist_id, credentials_file=filepath)

In [9]:
travis_scott = pd.DataFrame([i for i in genius_songs.keys()],columns=["pos"])
travis_scott.head()

,pos
0,100 Bottles
1,100 Bottles (Remix)
2,10 2 10
3,10 2 10 (Remix)
4,12 Disciples


In [10]:
genius_songs["90210"][1]

'https://genius.com/Travis-scott-90210-lyrics'

In [12]:
travis_scott["lyrics"] = spotfuncs.get_lyrics(genius_songs)

In [13]:
travis_scott_ = travis_scott.set_index("pos")

In [14]:
travis_scott_ = travis_scott_[travis_scott_["lyrics"] != ""]

In [66]:
travis_scott_["clean_lyrics"] = travis_scott_["lyrics"].apply(clean_lyrics)

In [67]:
print(travis_scott_.loc["90210","clean_lyrics"])

Dean, what's poppin'? Yo, Travis <N> Mhm, ooo-ooo-ooo-ooo-ooo <N> Mama's bailing down that road, craving 9021... <N> She a porn star girl, oh, from the valley <N> Honestly, God bless <N> Who left her hometown world all for the alley <N> Oh, created Lake Tahoe all from her panties <N> I hope it was wet like my jumper, though <N> Ooh, used to take the long way home <N> Long way home, all for that candy, mm <N> Baby's hooked on feeling numb <N> Do, do, do <N> Do, doo <N> Jacques turn La Flame, now he rolling on an Addy <N> 50 on a chain, another 50 on a Caddy, oh <N> He might pop him a pill, pop him a seal, pop anyone <N> Pop anything, pop anything to find that alley <N> Hmm, yeah, then find an alley, mm <N> Baby's hooked on feeling numb <N> Do, do, do <N> Do, doo <N> In the 90210, 90210, looking for that alley <N> In the 90210, 90210, looking for that alley, ooh <N> It's the superstar girl, superstar girl, roaming in that alley <N> Baby's hooked on feeling numb <N> In the 90210, 90210, s

`pickle` is a Python library that serializes Python objects to disk so that you can load them in later.

In [68]:
import pickle
pickle.dump(travis_scott_["clean_lyrics"].values, open("lyrics.pkl", "wb"))

# Question 2. Unigram Markov Chain Model

You will build a Markov chain for the artist whose lyrics you scraped in Lab A. Your model will process the lyrics and store the word transitions for that artist. The transitions will be stored in a dict called `chain`, which maps each word to a list of "next" words.

For example, if your song was ["The Joker" by the Steve Miller Band](https://www.youtube.com/watch?v=FgDU17xqNXo), `chain` might look as follows:

```
chain = {
    "some": ["people", "call", "people"],
    "call": ["me", "me", "me"],
    "the": ["space", "gangster", "pompitous", ...],
    "me": ["the", "the", "Maurice"],
    ...
}
```

Besides words, you should include a few additional states in your Markov chain. You should have `"<START>"` and `"<END>"` states so that we can keep track of how songs are likely to begin and end. You should also include a state called `"<N>"` to denote line breaks so that you can keep track of where lines begin and end. It is up to you whether you want to include normalize case and strip punctuation.

So for example, for ["The Joker"](https://www.azlyrics.com/lyrics/stevemillerband/thejoker.html), you would add the following to your chain:

```
chain = {
    "<START>": ["Some", ...],
    "Some": ["people", ...],
    "people": ["call", ...],
    "call": ["me", ...],
    "me": ["the", ...],
    "the": ["space", ...],
    "space": ["cowboy,", ...],
    "cowboy,": ["yeah", ...],
    "yeah": ["<N>", ...],
    "<N>": ["Some", ..., "Come"],
    ...,
    "Come": ["on", ...],
    "on": ["baby", ...],
    "baby": ["and", ...],
    "and": ["I'll", ...],
    "I'll": ["show", ...],
    "show": ["you", ...],
    "you": ["a", ...],
    "a": ["good", ...],
    "good": ["time", ...],
    "time": ["<END>", ...],
}
```

Your chain will be trained on not just one song, but by all songs by your artist.

In [93]:
lyrics[0]

"La Flame! <N> Hundred bottles, but I might need some more <N> Shot after shot, but I might need some more <N> Spendin' money, guala, might spend some more <N> Let's be honest, I just might make a switch <N> Hundred bottles, but I might need some more <N> Shot after shot, but I might need some more <N> Spendin' money, guala, might spend some more <N> Let's be honest, I just might make a switch <N> Let me in, let me in, let me in <N> Let me in, let me in, let me in <N> Straight up! <N> Hundred bottles, but I might need some more <N> Shot after shot, but I might need some more <N> Spendin' money, guala, might spend some more <N> Let's be honest, I just might make a switch <N> Let me in, let me in, let me in <N> Let me in, let me in, let me in <N> Let me in, let me in, let me in <N> Let me in, let me in, let me in <N> Let me in, let me in, let me in"

In [128]:
def train_markov_chain(songs):
    """
    Args:
      - lyrics: a list of strings, where each string represents
                the lyrics of one song by an artist.
    
    Returns:
      A dict that maps a single word ("unigram") to a list of
      words that follow that word, representing the Markov
      chain trained on the lyrics.
    """
    chain = {"<START>": []}
    for lyrics in songs:
        new_lyrics = lyrics.split(" ")
        chain["<START>"].append(new_lyrics[0])
        for i in range(len(new_lyrics)):
            if new_lyrics[i] not in chain:
                if i == len(new_lyrics) - 1:
                    chain[new_lyrics[i]] = ["<END>"]
                else:
                    chain[new_lyrics[i]] = [new_lyrics[i+1]]
            else:
                if i == len(new_lyrics) - 1:
                    chain[new_lyrics[i]].append("<END>")
                else:
                    chain[new_lyrics[i]].append(new_lyrics[i+1])
        
    return chain

In [130]:
# Load the pickled lyrics object that you created in Lab A.
import pickle
lyrics = pickle.load(open("lyrics.pkl", "rb"))

# Call the function you wrote above.
chain = train_markov_chain(lyrics)

# What words tend to start a song (i.e., what words follow the <START> tag?)
print(chain["<START>"][:20])

# What words tend to begin a line (i.e., what words follow the line break tag?)
print(chain["<N>"][:20])

['La', 'La', "Imma'", 'I', "I'm", 'I', 'Are', 'You', 'Part', 'Hustle', 'Bandana', 'Yeah', 'Yeah,', 'Yup', "Who's", 'Yo...', 'Dean,', 'Are', 'I', 'All']
['Hundred', 'Shot', "Spendin'", "Let's", 'Hundred', 'Shot', "Spendin'", "Let's", 'Let', 'Let', 'Straight', 'Hundred', 'Shot', "Spendin'", "Let's", 'Let', 'Let', 'Let', 'Let', 'Let']


Now, let's generate new lyrics using the Markov chain you constructed above. To do this, we'll begin at the `"<START>"` state and randomly sample a word from the list of words that follow `"<START>"`. Then, at each step, we'll randomly sample the next word from the list of words that followed each current word. We will continue this process until we sample the `"<END>"` state. This will give us the complete lyrics of a randomly generated song!

You may find the `random.choice()` function helpful for this question.

In [139]:
import random

def generate_new_lyrics(chain):
    """
    Args:
      - chain: a dict representing the Markov chain,
               such as one generated by generate_new_lyrics()
    
    Returns:
      A string representing the randomly generated song.
    """
    
    # a list for storing the generated words
    words = []
    # generate the first word
    words.append(random.choice(chain["<START>"]))
    
    start_word = words[0]
    while start_word != "<END>":
        next_word = random.choice(chain[start_word])
        words.append(next_word)
        start_word = next_word
        
    
    
    # join the words together into a string with line breaks
    lyrics = " ".join(words[:-1])
    return "\n".join(lyrics.split("<N>"))

In [140]:
print(generate_new_lyrics(chain))

Yeah, I shouldn't have a job 
 Bet-, bet-, bet-, bet- 
 I ain't playin' no paper it's just drive 
 ft. K. Swisha on Tylenol, having any of our way up, won't have the family, family 
 It's time to me, spend some Fedex Ayy, beat it bang, bang, yeah 
 Trap Sing McKnight 
 There 
 Take it down on feet 
 I'ma only of these niggas groupies on the time, I need that, flippin' that Mo City that smoke my residence, Godly feeling, dream chasers we're stuntin', but ive been through 
 Let’s get popped, my shine brighter days 
 I cannot handle mine, all the songs is southerners niggas think I disappeared in a tour, we'll have it up 
 You be like, everybody in the middle fingers up for him anything, pop it fall asleep this shit / and leave her ride that money all he never want jets 
 Squeeze the real, we didn't I really yah! I paid tryna have no 
 Pour some top when you know you're goin' crazy like I had to look like 
 Lil Uzi Vert - 
 oh 
 And she wanna have your skill 
 Bring your record from T 
 H

# Question 3. Bigram Markov Chain Model

Now you'll build a more complex Markov chain that uses the last _two_ words (or bigram) to predict the next word. Now your dict `chain` should map a _tuple_ of words to a list of words that appear after it.

As before, you should also include tags that indicate the beginning and end of a song, as well as line breaks. That is, a tuple might contain tags like `"<START>"`, `"<END>"`, and `"<N>"`, in addition to regular words. So for example, for ["The Joker"](https://www.azlyrics.com/lyrics/stevemillerband/thejoker.html), you would add the following to your chain:

```
chain = {
    (None, "<START>"): ["Some", ...],
    ("<START>", "Some"): ["people", ...],
    ("Some", "people"): ["call", ...],
    ("people", "call"): ["me", ...],
    ("call", "me"): ["the", ...],
    ("me", "the"): ["space", ...],
    ("the", "space"): ["cowboy,", ...],
    ("space", "cowboy,"): ["yeah", ...],
    ("cowboy,", "yeah"): ["<N>", ...],
    ("yeah", "<N>"): ["Some", ...],
    ("time", "<N>"): ["Come"],
    ...,
    ("<N>", "Come"): ["on", ...],
    ("Come", "on"): ["baby", ...],
    ("on", "baby"): ["and", ...],
    ("baby", "and"): ["I'll", ...],
    ("and", "I'll"): ["show", ...],
    ("I'll", "show"): ["you", ...],
    ("show", "you"): ["a", ...],
    ("you", "a"): ["good", ...],
    ("a", "good"): ["time", ...],
    ("good", "time"): ["<END>", ...],
}
```

In [ ]:
def train_markov_chain(lyrics):
    """
    Args:
      - lyrics: a list of strings, where each string represents
                the lyrics of one song by an artist.
    
    Returns:
      A dict that maps a tuple of 2 words ("bigram") to a list of
      words that follow that bigram, representing the Markov
      chain trained on the lyrics.
    """
    chain = {(None, "<START>"): []}
    for lyric in lyrics:
        # YOUR CODE HERE
        pass

    return chain

In [ ]:
# Load the pickled lyrics object that you created in Lab A.
import pickle
lyrics = pickle.load(open("lyrics.pkl", "rb"))

# Call the function you wrote above.
chain = train_markov_chain(lyrics)

# What words tend to start a song (i.e., what words follow the <START> tag?)
print(chain[(None, "<START>")])

Now, let's generate new lyrics using the Markov chain you constructed above. To do this, we'll begin at the `(None, "<START>")` state and randomly sample a word from the list of words that follow this bigram. Then, at each step, we'll randomly sample the next word from the list of words that followed the current bigram (i.e., the last two words). We will continue this process until we sample the `"<END>"` state. This will give us the complete lyrics of a randomly generated song!

In [ ]:
import random

def generate_new_lyrics(chain):
    """
    Args:
      - chain: a dict representing the Markov chain,
               such as one generated by generate_new_lyrics()
    
    Returns:
      A string representing the randomly generated song.
    """
    
    # a list for storing the generated words
    words = []
    # generate the first word
    words.append(random.choice(chain[(None, "<START>")]))
    
    # YOUR CODE HERE
    
    
    # join the words together into a string with line breaks
    lyrics = " ".join(words[:-1])
    return "\n".join(lyrics.split("<N>"))

In [ ]:
print(generate_new_lyrics(chain))

# Analysis

Compare the quality of the lyrics generated by the unigram model (in Lab B) and the bigram model (in Lab C). Which model seems to generate more reasonable lyrics? Can you explain why? What do you see as the advantages and disadvantages of each model?

**YOUR ANSWER HERE.**

# Submission Instructions

Once you are finished, follow these steps:

1. Restart the kernel and re-run this notebook from beginning to end by going to `Kernel > Restart Kernel and Run All Cells`.
2. If this process stops halfway through, that means there was an error. Correct the error and repeat Step 1 until the notebook runs from beginning to end.
3. Double check that there is a number next to each code cell and that these numbers are in order.

Then, submit your lab as follows:

1. Go to `File > Export Notebook As > PDF`.
2. Double check that the entire notebook, from beginning to end, is in this PDF file. (If the notebook is cut off, try first exporting the notebook to HTML and printing to PDF.)
3. Upload the PDF [to PolyLearn](https://polylearn.calpoly.edu/AY_2018-2019/mod/assign/view.php?id=349486).